In [ ]:
from __future__ import annotations

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep
import numpy as np

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-5, 5))

In [ ]:
from pathlib import Path

plot_dir = Path("plots/Limits/24Jun5")
plot_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
limits = [
    # 95%-, 68%-, 50%, 68%+, 95%+, observed
    [32.9, 45.9, 69.1, 113, 182, 142],
    [0.413, 0.585, 0.903, 1.48, 2.4, 1.06],
]

limits = np.array(limits)

observed_values = limits[:, 5]
expected_medians = limits[:, 2]
expected_68_lows = limits[:, 1]
expected_68_highs = limits[:, 3]
expected_95_lows = limits[:, 0]
expected_95_highs = limits[:, 4]

limit_labels = [
    r"$\sigma(pp \rightarrow HH)$ $(\kappa_{2V} = 1)$",
    r"$\sigma(pp \rightarrow qqHH)$ $(\kappa_{2V} = 0)$",
]


def format_limit(number):
    if number > 10:
        return f"{number:.0f}"  # No decimals
    else:
        return f"{number:.1f}"  # One decimal

In [ ]:
green = (0, 197 / 255.0, 0)
yellow = (255 / 255.0, 197 / 255.0, 0)
red = (255 / 255.0, 45 / 255.0, 45 / 255.0)

top_space = 0.6
limit_height = 0.4
y_lim = top_space + limit_height * len(observed_values)
dashed = (0, (2, 2))

xlims = [0.1, 250]


plt.rcParams.update({"text.usetex": False, "font.size": 24})

fig, ax = plt.subplots(figsize=(16, 7))

# Plot each set of limits
for i in range(len(observed_values)):
    y_base = y_lim - top_space - (i + 1) * limit_height
    y_rect = y_base
    y_line = y_base
    y_label = y_base + limit_height / 2

    # Add rectangles for the confidence intervals
    rect_95 = patches.Rectangle(
        (expected_95_lows[i], y_rect),
        expected_95_highs[i] - expected_95_lows[i],
        limit_height,
        linewidth=0,
        edgecolor="none",
        facecolor=yellow,
    )
    rect_68 = patches.Rectangle(
        (expected_68_lows[i], y_rect),
        expected_68_highs[i] - expected_68_lows[i],
        limit_height,
        linewidth=0,
        edgecolor="none",
        facecolor=green,
    )
    ax.add_patch(rect_95)
    ax.add_patch(rect_68)

    # Plot theory line
    ax.plot([1, 1], [y_line, y_line + limit_height], color=red, linewidth=2)
    # Plot the expected median line and observed value
    ax.plot(
        [expected_medians[i], expected_medians[i]],
        [y_line, y_line + limit_height],
        color="k",
        linewidth=2,
        linestyle=dashed,
    )
    ax.plot(
        [observed_values[i], observed_values[i]], [y_line, y_line + limit_height], "k-", linewidth=2
    )
    # Draw a small black circle at the center of the observed black line
    ax.plot(observed_values[i], y_line + limit_height / 2, "ko", markersize=8)

    # Set the y-axis label to the left of the y-axis
    textx = -0.28
    ax.text(
        textx,
        y_label + 0.06,
        limit_labels[i],
        verticalalignment="center",
        horizontalalignment="left",
        transform=ax.get_yaxis_transform(),
        fontsize=20,
    )

    ax.text(
        textx,
        y_label - 0.06,
        f"Expected: {format_limit(expected_medians[i])}\nObserved: {format_limit(observed_values[i])}",
        verticalalignment="center",
        horizontalalignment="left",
        transform=ax.get_yaxis_transform(),
        fontsize=16,
    )

    # Draw horizontal black lines above and below the limits
    ax.plot(xlims, [y_rect + limit_height, y_rect + limit_height], "k-", linewidth=2)
    if i == len(observed_values) - 1:
        ax.plot(xlims, [y_rect, y_rect], "k-", linewidth=2)

# Set the x and y axis labels
# ax.set_xlabel(r"95% CL limit on $\sigma(pp \rightarrow HH) / \sigma$")
ax.set_xlabel(r"95% CL limit on $\sigma / \sigma_\mathrm{Theory}$")
ax.set_yticks([])  # Remove y-axis ticks

# Set x-axis to logarithmic scale
ax.set_xscale("log")

# Add a legend in the top right with the specified order
legend_elements = [
    # plt.Line2D([0], [0], color=red, linewidth=2, label=r"$\sigma_\mathrm{Theory}$"),
    plt.Line2D(
        [0],
        [0],
        color="k",
        linewidth=2,
        label="Observed",
        marker="o",
        markersize=8,
        markerfacecolor="k",
        markeredgewidth=0,
    ),
    plt.Line2D([0], [0], color="k", linewidth=2, linestyle=dashed, label="Median expected"),
    patches.Patch(color=green, label="68% expected"),
    patches.Patch(color=yellow, label="95% expected"),
]
ax.legend(handles=legend_elements, loc="upper right", bbox_to_anchor=(1, 1), borderaxespad=0.8)

# Add common kappa values in the top left of the figure box
common_kappa_text = r"$\kappa_{\lambda} = \kappa_{t} = \kappa_{V} = 1$"
ax.text(
    0.03,
    0.94,
    common_kappa_text,
    transform=ax.transAxes,
    fontsize=20,
    verticalalignment="top",
    horizontalalignment="left",
)

# Set y-axis limits to remove the space below the bottom limit
ax.set_ylim(0, y_lim)

# Set x-axis limits
ax.set_xlim(xlims)

# Configure the x-axis to avoid scientific notation
ax.xaxis.set_major_formatter(plt.ScalarFormatter())
ax.xaxis.get_major_formatter().set_scientific(False)
ax.xaxis.set_major_formatter(mticker.FormatStrFormatter("%g"))  # plot decimals as needed

hep.cms.label(label=None, ax=ax, data=True, lumi=138, com=13)
plt.tight_layout()
plt.savefig(plot_dir / "limits.pdf", bbox_inches="tight")
plt.show()